# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json','r') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [3]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'],timeout_s=5.0)

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'coffee'

In [5]:
location.split(',')[0]

'Seattle'

In [6]:
## Specify fodler for saving data

FOLDER = 'Data/'
os.makedirs(FOLDER,exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
#JSON_FILE = 'Seattle-Pizza'
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Seattle-coffee.json'

### Check if Json File exists and Create it if it doesn't

### Load JSON FIle and account for previous results

In [8]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term =term,location = location)

In [10]:
## Check if JSON_FILE exists

## If it does not exist: 
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only

    
    ## If JSON_FILE included a folder:

        # create the folder

        
        
    ## INFORM USER AND SAVE EMPTY LIST

    
    
    ## save the first page of results

        
## If it exists, inform user


In [11]:
#type(results)
results

{'businesses': [{'id': 'cpfool0I6kNLpIe0Su6fqw',
   'alias': 'avole-ethiopian-coffee-seattle',
   'name': 'Avole Ethiopian Coffee',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Wg18Mz7hk7acZIR1jJQoGA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/avole-ethiopian-coffee-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
   'review_count': 10,
   'categories': [{'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'},
    {'alias': 'coffeeroasteries', 'title': 'Coffee Roasteries'},
    {'alias': 'coffee', 'title': 'Coffee & Tea'}],
   'rating': 5.0,
   'coordinates': {'latitude': 47.61309227798837,
    'longitude': -122.3018459481764},
   'transactions': [],
   'location': {'address1': '2330 E Union St',
    'address2': '',
    'address3': None,
    'city': 'Seattle',
    'zip_code': '98122',
    'country': 'US',
    'state': 'WA',
    'display_address': ['2330 E Un

In [12]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
results['businesses'][0]

{'id': 'cpfool0I6kNLpIe0Su6fqw',
 'alias': 'avole-ethiopian-coffee-seattle',
 'name': 'Avole Ethiopian Coffee',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/Wg18Mz7hk7acZIR1jJQoGA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/avole-ethiopian-coffee-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
 'review_count': 10,
 'categories': [{'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'},
  {'alias': 'coffeeroasteries', 'title': 'Coffee Roasteries'},
  {'alias': 'coffee', 'title': 'Coffee & Tea'}],
 'rating': 5.0,
 'coordinates': {'latitude': 47.61309227798837,
  'longitude': -122.3018459481764},
 'transactions': [],
 'location': {'address1': '2330 E Union St',
  'address2': '',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['2330 E Union St', 'Seattle, WA 98122']},
 'phone': '',
 'display_ph

In [14]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,cpfool0I6kNLpIe0Su6fqw,avole-ethiopian-coffee-seattle,Avole Ethiopian Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/Wg18Mz...,False,https://www.yelp.com/biz/avole-ethiopian-coffe...,10,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",5.0,"{'latitude': 47.61309227798837, 'longitude': -...",[],"{'address1': '2330 E Union St', 'address2': ''...",,,461.839660,NaN
1,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,91,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877,$
2,Q8Nlx_FeZ4LW0kdbnHoyrw,qed-coffee-seattle,QED Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/HwagNI...,False,https://www.yelp.com/biz/qed-coffee-seattle?ad...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.5896816513796, 'longitude': -1...",[delivery],"{'address1': '1418 31st Ave S', 'address2': ''...",+12062357678,(206) 235-7678,2615.675293,$$
3,FVzl8rDPiTWEtrNEuCu-Xg,storyville-coffee-company-seattle-9,Storyville Coffee Company,https://s3-media3.fl.yelpcdn.com/bphoto/TecGMk...,False,https://www.yelp.com/biz/storyville-coffee-com...,2079,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.60895949363687, 'longitude': -...","[delivery, pickup]","{'address1': '94 Pike St', 'address2': 'Ste 34...",+12067805777,(206) 780-5777,3385.734739,$$
4,2ZrovOWUNfzpSJlx-7DYCw,hello-em-viet-coffee-and-roastery-seattle-2,Hello Em Viet Coffee & Roastery,https://s3-media2.fl.yelpcdn.com/bphoto/Wua0EA...,False,https://www.yelp.com/biz/hello-em-viet-coffee-...,252,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...",4.5,"{'latitude': 47.59732, 'longitude': -122.31544}",[],"{'address1': '1227 S Weller St', 'address2': '...",,,2296.777516,$
5,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,109,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62622241003878, 'longitude': -...","[delivery, pickup]","{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1487.954980,$
6,n9NPzpLTl0iOBH3R4Zk6yg,pike-street-coffee-seattle,Pike Street Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/P7UkSx...,False,https://www.yelp.com/biz/pike-street-coffee-se...,4,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 47.608397612504156, 'longitude': ...",[],"{'address1': '1501 Western Ave', 'address2': '...",+12068299594,(206) 829-9594,3480.549009,NaN
7,9TnUll8eOc5FREIQqrcrFg,verite-coffee-seattle,Verite Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/SaSFeE...,False,https://www.yelp.com/biz/verite-coffee-seattle...,38,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 47.61175, 'longitude': -122.28953}",[],"{'address1': '1101 34th Ave', 'address2': '', ...",+12067094497,(206) 709-4497,483.915129,$
8,KkDoL5MCOMAPJA4NaS-xgw,fuel-coffee-and-books-seattle,Fuel Coffee & Books,https://s3-media1.fl.yelpcdn.com/bphoto/kx3cc_...,False,https://www.yelp.com/biz/fuel-coffee-and-books...,101,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.0,"{'latitude': 47.63967, 'longitude': -122.30182}",[delivery],"{'address1': '2300 24th Ave E', 'address2': No...",+12063280700,(206) 328-0700,2988.904316,$
9,4o7FQKh8GXSOjoJa1OXfsw,overcast-coffee-company-seattle,Overcast Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/xz55iC...,False,https://www.yelp.com/biz/overcast-coffee-compa...,41,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 47.62092, 'longitude': -122.31256}","[delivery, pickup]","{'address1': '312 15th Ave E', 'address2': '',...",+12063222220,(206) 322-2220

In [15]:
## How many results total?
results['total']

1400

In [16]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [17]:
results_per_page = len(results['businesses'])

In [18]:
results_per_page

20

- Where is the actual data we want to save?

In [19]:
# ## How many did we get the details for?
# results_per_page = None
# results_per_page

- Calculate how many pages of results needed to cover the total_results

In [20]:
## How many results total?
# Use math.ceil to round up for the total number of pages of results.
import math
n_pages= math.ceil(results['total']/results_per_page)
n_pages

70

In [21]:
## Check if JSON_FILE exists

## If it does not exist: 
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
if os.path.isfile(JSON_FILE)==False:
    
     # file does not exist so print message and create empty file
    print(" The file does not exist. Creating empty file.")
    with open(JSON_FILE,'w') as f:
        json.dump(results['businesses'],f)
## If it exists, inform user
else:
    print('File exists')

File exists


In [22]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
        
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE) as f:
            prev_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(prev_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(term = term,location=location,offset=n_results)

        ## append new results and save to file
        
        prev_results.extend(results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(prev_results,f)
        
       
        


  0%|          | 0/70 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

## Open the Final JSON File with Pandas

In [ ]:
df = pd.read_json(JSON_FILE)
df

In [ ]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

In [ ]:
## Save it as a compressed csv (to save space)


## Bonus: compare filesize with os module's `os.path.getsize`

In [ ]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

## Next Class: Processing the Results and Mapping 